<a href="https://colab.research.google.com/github/YoshiyukiKono/gen_ai-sandbox/blob/main/langchain_few_shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install faiss-cpu
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 112.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.7 MB/s eta 0:00:00


In [2]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)
import openai
openai.api_key = 'your-api-key-here'
# few shot promptingの定義
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {"input": "Write me a poem about the moon",
     "output": "One for the moon, and one for me, who are we to talk about the moon?",},
]

# データの入力と出力を連結してベクトル化のためのリストを作成
to_vectorize = [" ".join(example.values()) for example in examples]

# OpenAIの埋め込みモデルを初期化
embeddings = OpenAIEmbeddings()

# 教師データをベクトル化してChromaベクトルストアに保存
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

# セマンティック類似性に基づく選択器を初期化。kは類似する最上位kの例を選択する数を意味する
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# メッセージプロンプトテンプレートを定義
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

# 最終的なチャットプロンプトテンプレートを定義
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# チャットモデルとプロンプトテンプレートをチェーン化
chain = final_prompt | ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo")

# チェーンを使って質問を投げ、結果を表示
print(chain.invoke({"input": "What's 3+3?"}).content)

ValidationError: ignored

https://zenn.dev/tsuzukia/articles/8fc74bdb8770a5